In [2]:
1973/22.5

87.68888888888888

In [3]:
75*3+89*1.5+90*3+93*3+95*4+99*1+86*3+92*1+88*3+85*1`

2085.5

In [4]:
2085.5/23.5

88.74468085106383

In [5]:
88*3+92*1.5+86*3+97*3+91*4+99*1+81*3+88*1+82*3+89*1

2080.0

In [6]:
2080/23.5

88.51063829787235

In [7]:
78*3+90*1.5+81*3+98*3+94*4+93*3+86*1+79*3+87*1

1971.0

In [8]:
1971/22.5

87.6

In [3]:
import torch.nn as nn
kernel_sizes=(5, 9, 13)
m = nn.ModuleList(
            [nn.MaxPool2d(kernel_size=ks, stride=1, padding=ks // 2) for ks in kernel_sizes]
        )
print(m)

ModuleList(
  (0): MaxPool2d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (1): MaxPool2d(kernel_size=9, stride=1, padding=4, dilation=1, ceil_mode=False)
  (2): MaxPool2d(kernel_size=13, stride=1, padding=6, dilation=1, ceil_mode=False)
)


In [5]:
class SPPBottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_sizes=(5, 9, 13), act="silu"):
        super(SPPBottleneck, self).__init__()
        hidden_channels = in_channels // 2
        self.conv1 = BaseConv(in_channels, out_channels, 1, stride=1, act=act)
        self.m = nn.ModuleList(
            [nn.MaxPool2d(kernel_size=ks, stride=1, padding=ks // 2) for ks in kernel_sizes]
        )
        conv2_channels = hidden_channels * (len(kernel_sizes) + 1)
        self.conv2 = BaseConv(conv2_channels, out_channels, 1, stride=1, act=act)

    def forward(self, x):
        x = self.conv1(x)
        out = torch.cat([x] + [m(x) for m in self.m], dim=1)
        y=[m(x) for m in self.m]
        print(y[0].size())
        return self.conv2(out)


In [9]:
def get_activation(name="silu", inplace=True):
    if name == "silu":
        module = nn.SiLU(inplace=inplace)
    elif name == "relu":
        module = nn.ReLU(inplace=inplace)
    elif name == "lrelu":
        module = nn.LeakyReLU(0.1, inplace=inplace)
    else:
        raise AttributeError("Unsupported act type: {}".format(name))
    return module

In [7]:
class BaseConv(nn.Module):
    def __init__(self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"):
        super(BaseConv, self).__init__()
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = get_activation(act)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))

In [10]:
    from cv2 import cv2
    import numpy as np

    net = SPPBottleneck(3, 3, 1)
    img = cv2.imread("dog.jpg")
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).to(torch.float32)
    img = torch.unsqueeze(img, 0)
    img = net(img)
    img = torch.squeeze(img, 0)
    img = img.detach().numpy()
    img = img * 255 / img.max()
    img = np.transpose(img, (1, 2, 0))
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #无需更改通道

    cv2.imshow("test", img)
    cv2.waitKey()
    print(img.shape)


TypeError: 'int' object is not iterable

In [13]:
import torch
import torch.nn as nn
BN = nn.BatchNorm1d(100)
input = torch.randn(20, 100)
output = m(input)


NotImplementedError: 